<a href="https://colab.research.google.com/github/Fadhaa/Currencies-Price-prediction/blob/main/stat_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Install the required tools*

In [132]:
!pip -q install git+https://github.com/huggingface/transformers
!pip -q install git+https://github.com/huggingface/transformers.git -U accelerate
!pip install huggingface_hub
!pip install ipywidgets


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [133]:
from huggingface_hub import notebook_login

# Login into Huggingface with API key

In [134]:
notebook_login()

In [166]:
# Import necessary libraries
import gc
import math
import re
import sys
import subprocess
import numpy as np
import pandas as pd
import random

In [167]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    StoppingCriteria,
    set_seed
)

In [168]:
import transformers
print(f"Transformers Version: {transformers.__version__}")
set_seed(14)

Transformers Version: 4.43.0.dev0


# The model that we might use as a tool for combining LLM and Statistics is( gemma-1.1-2b-it)

In [139]:
MODEL_PATH = "google/gemma-1.1-2b-it"##this is model path
DEEP = True

config = AutoConfig.from_pretrained(MODEL_PATH)##pretrain the model
config.gradient_checkpointing = True

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

device_map = [('model.embed_tokens', 0),
              ('model.layers.0', 0),
              ('model.layers.1', 0),
              ('model.layers.2', 0),
              ('model.layers.3', 0),
              ('model.layers.4', 0),
              ('model.layers.5', 0),
              ('model.layers.6', 0),
              ('model.layers.7', 0),
              ('model.layers.8', 0),
              ('model.layers.9', 0),
              ('model.layers.10', 0),
              ('model.layers.11', 0),
              ('model.layers.12', 0),
              ('model.layers.13', 0),
              ('model.layers.14', 0),
              ('model.layers.15', 0),
              ('model.layers.16', 0),
              ('model.layers.17', 0),
              ('model.layers.18', 0),
              ('model.layers.19', 0),
              ('model.layers.20', 0),
              ('model.layers.21', 0),
              ('model.layers.22', 1),
              ('model.layers.23', 1),
              ('model.layers.24', 1),
              ('model.layers.25', 1),
              ('model.layers.26', 1),
              ('model.layers.27', 1),
              ('model.layers.28', 1),
              ('model.layers.29', 1),
              ('model.norm', 1),
              ('lm_head', 1)]


In [140]:
device_map = {ii:jj for (ii,jj) in device_map}

In [141]:
import torch

In [142]:
torch.cuda.is_available()

True

# Loading the Model

In [143]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map='auto',
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
    )

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype='auto',
    device_map='auto',
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [144]:
xi= np.random.normal(0, 1, 30)# generat 30 observations from N(0,1)

In [169]:
# generat dependent variable
yi=0.5+2*xi

In [170]:
x_str=','.join([str(z) for z in xi])

In [173]:
y_str=','.join([str(z) for z in yi])

In [172]:
# define the instructing prompt
prompt="for the data the dependent variable y is :"+y_str+ " and the independent variable x is :"+x_str+" find the maximum likelihood estimators of the parameters of the linear regression model of one variable and intercept by using statsmodels. do not solve it. Only generate python code that give the final results ?"

In [150]:
def process_code(code, return_shell_output=False):

    def repl(match):
        if "real" not in match.group():
            return "{}{}".format(match.group()[:-1], ', real=True)')
        else:
            return "{}{}".format(match.group()[:-1], ')')
    code = re.sub(r"symbols\([^)]+\)", repl, code)

    if return_shell_output:
        code = code.replace('\n', '\n    ')
            # Add a try...except block
        code = "\ntry:\n    from sympy import *\n{}\nexcept Exception as e:\n    print(e)\n    print('FAIL')\n".format(code)

    if not return_shell_output:
        print(code)
    with open('code.py', 'w') as fout:
        fout.write(code)

    batcmd = 'timeout 7 ' + sys.executable + ' code.py'
    try:
        shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
        return_value = return_last_print(shell_output, -1)
        print(shell_output)
        if return_shell_output:
            if return_value=='FAIL':
                CODE_STATUS = False
                return_value = return_last_print(shell_output, -2)
                if "not defined" in return_value:
                    return_value+='\nTry checking the formatting and imports'
            else:
                CODE_STATUS = True
            return return_value, CODE_STATUS
        code_output = round(float(eval(return_value))) % 1000
    except Exception as e:
        print(e,'shell_output')
        code_output = -1

    if return_shell_output:
        if code_output==-1:
            CODE_STATUS = False
        else:
            CODE_STATUS = True
        return code_output, CODE_STATUS


    return code_output

In [152]:
from collections import defaultdict

In [154]:
import re
from collections import defaultdict
from collections import Counter

from numpy.random import choice
import numpy as np

temperature = 0.9
top_p = 1.0

temperature_coding = 0.9
top_p_coding = 1.0

bbb=[]
total_results = {}
total_answers = {}
best_stats = {}
total_outputs = {}
question_type_counts = {}
starting_counts = (2,3)
stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"]


In [155]:
problem =prompt

In [28]:
problem

'for the data the dependent variable y is :3.602678222506126,0.658372049217643,0.8479530569123397,0.35532686766764654,-3.508658800344371,0.7893556263806322,-2.5023372445941905,0.9222189019020861,-0.6164101107521522,2.6690581297676106,0.12742175195198274,0.5293227495938443,-1.6511389388733662,1.7845041484337525,0.13932657734313764,1.7406005067503543,2.121948669236295,2.055871732795083,0.02502064411553584,-3.2838246906154196,-1.844243587783275,2.1724139426958806,2.5185468006643528,3.0861987218193314,-0.5986623311224653,-1.4634257299582547,2.3293455157704033,2.2461407565537512,0.5442222834102208,0.4917128329546021 and the independent variable x is :1.551339111253063,0.07918602460882154,0.17397652845616982,-0.07233656616617674,-2.0043294001721854,0.14467781319031608,-1.5011686222970952,0.21110945095104303,-0.5582050553760761,1.0845290648838053,-0.18628912402400863,0.014661374796922139,-1.075569469436683,0.6422520742168762,-0.18033671132843118,0.6203002533751771,0.8109743346181475,0.7779358

In [156]:
draw = """Below is a stat problem to generate python code to solve it:
\"{}\"
generate only python code by use python statsmodel that can be applied to given data\n\n.

Approach:"""


In [157]:
initail_message = draw.format(problem," \n")

In [158]:
pro = f"User: {initail_message}"

In [159]:
model_inputs = tokenizer(pro, return_tensors='pt').to(model.device)

In [160]:
input_len = len(model_inputs['input_ids'][0])

In [161]:
USE_PAST_KEY=True
total_token=2040
temperature=0.9
top_p=1

In [163]:
import torch
from transformers import StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            last_token = input_ids[0][-len(stop):]
            if torch.all(torch.eq(stop,last_token)):

                return True
        return False


stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"] #,
stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [80]:
generation_output = model.generate(**model_inputs,
                                   max_new_tokens=total_token,
                                   return_dict_in_generate=USE_PAST_KEY,
                                   do_sample = True,
                                   temperature = temperature,
                                   top_p = top_p,
                                   num_return_sequences=1)######, stopping_criteria = stopping_criteria)

In [164]:
if USE_PAST_KEY:
  output_ids = generation_output.sequences[0]
else:
  output_ids = generation_output[0]

In [82]:
decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)

# the final outcome from gemma is the python code that can be used for parameters implementation

In [83]:
print(decoded_output)

User: Below is a stat problem to generate python code to solve it:
"for the data the dependent variable y is :3.602678222506126,0.658372049217643,0.8479530569123397,0.35532686766764654,-3.508658800344371,0.7893556263806322,-2.5023372445941905,0.9222189019020861,-0.6164101107521522,2.6690581297676106,0.12742175195198274,0.5293227495938443,-1.6511389388733662,1.7845041484337525,0.13932657734313764,1.7406005067503543,2.121948669236295,2.055871732795083,0.02502064411553584,-3.2838246906154196,-1.844243587783275,2.1724139426958806,2.5185468006643528,3.0861987218193314,-0.5986623311224653,-1.4634257299582547,2.3293455157704033,2.2461407565537512,0.5442222834102208,0.4917128329546021 and the independent variable x is :1.551339111253063,0.07918602460882154,0.17397652845616982,-0.07233656616617674,-2.0043294001721854,0.14467781319031608,-1.5011686222970952,0.21110945095104303,-0.5582050553760761,1.0845290648838053,-0.18628912402400863,0.014661374796922139,-1.075569469436683,0.6422520742168762,-